In [1]:
from databroker.v2 import temp
from bluesky import RunEngine
from bluesky.plans import count
from bluesky.plan_stubs import mv, monitor, unmonitor
from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device
import asyncio
import random



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det, motor

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = await SECoP_Node_Device.create('192.168.15.3','10770',RE.loop)

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")






sample_changer.HZB ready


('c574854c-a6fa-47bb-ad4d-5a2fbcb8dd4a',)

In [9]:
await robo_dev.read_configuration()

storage:SECoPReadableDevice= robo_dev.storage

reading = await storage.value.read()

loadshort:SECoP_CMD_Device = storage.load_short_dev


reading = await storage.value.read()
print(reading)


{'sample_changer-HZB-storage-value': {'value': ["{'sample_name': 'NugatRittersport', 'substance': 'Nugat', 'substance_code': '4', 'sample_pos': 1, 'manufacturer': 'Rittersport', 'sample_id': 'bfc4e3fd-83c3-477b-af5b-cbd66e147a2b', 'color': 'dark blue', 'mass': 0.0167}"], 'timestamp': 1693403926.342577}}


In [14]:


loadshort:SECoP_CMD_Device = storage.load_short_dev

for samplepos in range(2,3):
    await loadshort.samplepos_arg.set(samplepos)
    await loadshort.substance_arg.set(random.randint(0,6))

    await loadshort.load_short_x.execute()
    

    await storage.wait_for_IDLE()
    await asyncio.sleep(4)
    print(samplepos)
        
        







2


rxthread ended with ConnectionResetError(104, 'Connection reset by peer')
try to reconnect to 192.168.15.3:10770
can not connect to 192.168.15.3:10770, timed out
can not reconnect to sample_changer.HZB (CommunicationFailed('can not connect to 192.168.15.3:10770, timed out'))
continue trying to reconnect


In [4]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

RE(count([storage],delay=.2,num=10))



('8efc9d53-a033-448a-9bfe-b284d29f2c09',)

In [10]:
sample:SECoPMoveableDevice = robo_dev.sample


for i in range(1,2):
    #grab sample 1 and move it into the measurement Pos
    RE(mv(sample,1))
    
    # Start measuring program (rotates Sample; sample module stwitches to busy state)
    await sample.measure_dev.measure_x.execute()
    
    # RE(monitor([det]))??
    # Start measuring with Sensor/Detector 

    # measure CMD ends when module goes back to IDLE    
    await sample.wait_for_IDLE()
    #RE(unmonitor([det]))??
    
    # put sample back into storage
    RE(mv(sample,0))